In [1]:
data_var = '2024-01-17'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10606,2024-01-17,Brasil Nbb,19:30,Brasília,Minas,6.85,1.08,157.5,1.82,1.85,10.5,2.08,1.63,h0EtqE96,0.145985,0.925926,0.549451,0.540541,0.071911,0.4,0.4,NaN,NaN,372.488,205.768582,0.552417,115.156,32.042617,0.278254,180.938,122.572,161.20,121.03,0.0,0.0,0.0,0.0,1.029005,0.011560,0.171535,-2.81,-0.562,-10.409253,0.000000,0.0,0.000000,0.67,0.134,0.597015,0.000000,0.0,0.000000
10607,2024-01-17,Eua Nba,00:00,Los Angeles Clippers,Oklahoma City Thunder,1.47,2.98,236.5,1.86,2.07,-6.5,2.06,1.87,OfTGne2Q,0.680272,0.335570,0.537634,0.483092,0.015843,0.4,0.6,NaN,NaN,184.482,29.659644,0.160773,171.258,30.016672,0.175272,166.562,238.184,151.20,191.10,0.0,0.0,0.0,0.0,0.479879,0.075569,0.068372,1.75,0.350,1.342857,0.684239,0.9,0.215761,-0.43,-0.086,-23.023256,0.543791,0.4,-0.143791
10608,2024-01-17,Eua Nba,21:00,Detroit Pistons,Minnesota Timberwolves,5.85,1.14,220.5,1.86,1.94,10.5,2.06,1.76,Q3FrrFAs,0.170940,0.877193,0.537634,0.515464,0.048133,0.4,0.2,NaN,NaN,628.698,582.921632,0.927189,190.674,31.013534,0.162652,352.860,203.570,408.10,175.15,1.0,0.0,0.0,0.0,0.952925,0.029773,0.111064,-2.60,-0.520,-9.326923,0.338766,0.1,-0.238766,-1.81,-0.362,-0.386740,0.556528,0.6,0.043472
10609,2024-01-17,Eua Nba,21:30,Atlanta Hawks,Orlando Magic,1.69,2.36,231.5,1.83,2.05,-4.5,2.06,1.86,fwKnsZPm,0.591716,0.423729,0.546448,0.487805,0.015445,0.4,0.4,NaN,NaN,173.690,42.883603,0.246897,337.130,165.464800,0.490804,189.204,451.882,146.06,334.18,0.0,0.0,0.0,0.0,0.233956,0.080187,0.072154,0.00,0.000,inf,0.602191,0.4,-0.202191,4.19,0.838,1.622912,0.364392,0.4,0.035608
10610,2024-01-17,Eua Nba,21:30,Cleveland Cavaliers,Milwaukee Bucks,2.54,1.61,236.5,1.86,2.06,3.5,2.03,1.89,Gh8euDf0,0.393701,0.621118,0.537634,0.485437,0.014819,0.4,0.4,NaN,NaN,163.928,22.726669,0.138638,179.004,30.000229,0.167595,163.928,191.290,173.31,165.24,0.0,0.0,0.0,0.0,0.316920,0.072154,0.050508,1.98,0.396,3.888889,0.642138,0.8,0.157862,-0.59,-0.118,-5.169492,0.620258,0.5,-0.120258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10746,2024-01-17,Eua Ncaa,23:00,North Carolina,Louisville,1.04,13.50,155.5,1.91,1.91,-22.5,2.05,1.71,rV8TszJt,0.961538,0.074074,0.523560,0.523560,0.035613,0.8,0.0,NaN,NaN,123.962,16.916300,0.136464,456.688,203.032426,0.444576,121.930,0.000,115.36,744.00,0.0,1.0,0.0,0.0,1.211905,0.000000,0.127881,0.01,0.002,20.000000,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10747,2024-01-17,Europa Liga Letônia,13:30,TAL TECH,Rigas Zelli,2.76,1.41,154.5,1.84,1.84,4.5,2.00,1.72,rytQKJhf,0.362319,0.709220,0.543478,0.543478,0.071539,0.2,0.6,NaN,NaN,298.946,235.230911,0.786868,231.084,119.738528,0.518160,184.250,149.322,111.22,89.89,0.0,0.0,0.0,0.0,0.457839,0.000000,0.106446,-1.98,-0.396,-4.444444,0.000000,0.0,0.000000,-1.02,-0.204,-2.009804,0.000000,0.0,0.000000
10748,2024-01-17,Europa Wbbl Feminina,11:30,Aistes LSU Kaunas F,RSU Merks F,1.56,2.32,132.5,1.85,1.85,-3.5,1.85,1.85,SMbcNM4P,0.641026,0.431034,0.540541,0.540541,0.072060,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,420.00,0.00,1.0,0.0,0.0,0.0,0.277011,0.000000,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10749,2024-01-17,Finlândia Divisão I Feminina,12:30,Puhuttaret F,Pyrinto II F,2.31,1.57,132.5,1.85,1.85,3.5,1.83,1.87,W8p2SZBr,0.432900,0

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Clippers,Oklahoma City Thunder,236.5,1.86,1.0000,Over
1,21:30,Eua Nba,Atlanta Hawks,Orlando Magic,231.5,1.83,0.9999,Over
2,21:30,Eua Nba,Cleveland Cavaliers,Milwaukee Bucks,236.5,1.86,1.0000,Over
3,23:00,Eua Nba,Utah Jazz,Golden State Warriors,242.5,1.81,0.9999,Over
4,22:00,Eua Nba,New Orleans Pelicans,Charlotte Hornets,228.5,1.82,0.8301,Over
